In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

from surv_ai.lib.log import logger, AgentLogLevel
import logging

logger.set_log_level(AgentLogLevel.INTERNAL)
logging.basicConfig(level=logging.INFO)

In [3]:
from surv_ai import (
    GPTClient,
    Survey,
    ToolBelt,
    GoogleCustomSearchTool,
    Knowledge
)

client = GPTClient(os.environ["OPEN_AI_API_KEY"])

tool_belt = ToolBelt(
    tools=[
        GoogleCustomSearchTool(
            google_api_key=os.environ["GOOGLE_API_KEY"],
            google_search_engine_id=os.environ["GOOGLE_SEARCH_ENGINE_ID"],
            start_date="2023-01-01",
            end_date="2023-05-01",
            n_pages=10,
        )
    ]
)
base_knowledge = [
    Knowledge(
        text="It is currently 2023/05/01, all the articles are from 2023.",
        source="Additional context",
    ),
]

survey = Survey(
    client=client,
    tool_belt=tool_belt,
    base_knowledge=base_knowledge,
    max_knowledge_per_agent=3,
    n_agents=10,
)

await survey.conduct(
    "California experienced a significant amount of rainfall this winter.",
)  # This should always returns high a confidence agreement.

INFO:surv_ai:...Using tool: SEARCH("California rainfall winter 2023")...
INFO:surv_ai:......Retrieving www.nytimes.com article with title A Very Wet Winter Has Eased California’s Drought, but Water Woes Remain......
INFO:surv_ai:......Retrieving www.nytimes.com article with title How Do California’s Storms Weigh In Compared With History’s Big Ones?......
INFO:surv_ai:......Retrieving Washington Post article with title New winter storm could push California’s snowpack to record high......
INFO:surv_ai:......Retrieving www.nytimes.com article with title What Does All This Rain Mean for California’s Drought?......
INFO:surv_ai:......Retrieving CNN article with title Snowfall tops 6.5 feet and rainfall tops 5 inches across southern California | CNN......
INFO:surv_ai:......Retrieving Washington Post article with title The science behind California’s extremely wet winter, in maps......
INFO:surv_ai:......Retrieving Washington Post article with title In California, a drought turned to floods

SurveyResponse(in_favor=10, against=0, undecided=0, error=0, percent_in_favor=1.0, uncertainty=0.0)

In [6]:
from surv_ai import (
    GPTClient,
    Survey,
    ToolBelt,
    GoogleCustomSearchTool,
    Knowledge
)

client = GPTClient(os.environ["OPEN_AI_API_KEY"])

tool_belt = ToolBelt(
    tools=[
        GoogleCustomSearchTool(
            google_api_key=os.environ["GOOGLE_API_KEY"],
            google_search_engine_id=os.environ["GOOGLE_SEARCH_ENGINE_ID"],
            start_date="2023-01-01",
            end_date="2023-05-01",
            n_pages=10,
        )
    ]
)
base_knowledge = [
    Knowledge(
        text="It is currently 2023/05/01, all the articles are from 2023.",
        source="Additional context",
    ),
]

survey = Survey(
    client=client,
    tool_belt=tool_belt,
    base_knowledge=base_knowledge,
    max_knowledge_per_agent=3,
    n_agents=10,
)

await survey.conduct(
    "California experienced little rainfall this winter.",
)  # This should always returns a high confidence disagreement.

INFO:surv_ai:...Using tool: SEARCH("California little rainfall winter 2023")...
INFO:surv_ai:......Retrieving www.nytimes.com article with title A Very Wet Winter Has Eased California’s Drought, but Water Woes Remain......
INFO:surv_ai:......Retrieving CNN article with title Record snowpack, nearly full reservoirs: Here's the state of California's drought after an epic winter | CNN......
INFO:surv_ai:......Retrieving Washington Post article with title In California, a drought turned to floods. Forecasters didn’t see it coming.......
INFO:surv_ai:......Retrieving www.nytimes.com article with title What Does All This Rain Mean for California’s Drought?......
INFO:surv_ai:......Retrieving www.nytimes.com article with title How Climate Change Is Shaping California’s Winter Storms......
INFO:surv_ai:......Retrieving Washington Post article with title The science behind California’s extremely wet winter, in maps......
INFO:surv_ai:......Retrieving CNN article with title Snowfall tops 6.5 fee

SurveyResponse(in_favor=2, against=8, undecided=0, error=0, percent_in_favor=0.2, uncertainty=0.0)

In [3]:
from surv_ai import (
    GPTClient,
    Model,
    ToolBelt,
    GoogleCustomSearchTool,
    Knowledge,
    Survey,
    SurveyParameter
)


client = GPTClient(os.environ["OPEN_AI_API_KEY"])

def build_parameter(date_range: tuple[str, str]):
    tool_belt = ToolBelt(
        tools=[
            GoogleCustomSearchTool(
                google_api_key=os.environ["GOOGLE_API_KEY"],
                google_search_engine_id=os.environ["GOOGLE_SEARCH_ENGINE_ID"],
                n_pages=20,
                start_date=date_range[0],
                end_date=date_range[1]
            ),
        ],
    )
    base_knowledge = [
        Knowledge(
            text=f"It is currently {date_range[0]}. The included articles were published between {date_range[0]} and {date_range[1]}",
            source="Additional context",
        ),
    ]
    return SurveyParameter(
        independent_variable=date_range[1],
        kwargs={
            "client": client,
            "n_agents": 100,
            "max_knowledge_per_agent":20,
            "max_concurrency": 10,
            "tool_belt": tool_belt,
            "base_knowledge": base_knowledge,
        },
    )

date_ranges = [
    ('2022-05-01', '2022-06-01'),
    ('2022-06-01', '2022-07-01'),
    ('2022-07-01', '2022-08-01'),
    ('2022-08-01', '2022-09-01'),
    ('2022-09-01', '2022-10-01'),
    ('2022-10-01', '2022-11-05'),
]

model = Model(
    Survey,
    parameters=[build_parameter(date_range) for date_range in date_ranges],
)

results = await model.build(
    "Democrats are favored to maintain control of the Senate in the 2022 November Midterm elections.",
)

INFO:surv_ai:...Using tool: SEARCH("2022 Senate midterm election predictions")...
INFO:surv_ai:......Retrieving CNN article with title The 10 Senate seats most likely to flip in 2022 | CNN Politics......
INFO:surv_ai:......Retrieving NBC News article with title Primary election 2022 live updates: Pa. GOP Senate race in dead heat......
INFO:surv_ai:......Retrieving Washington Post article with title A decision to overturn Roe v. Wade might upend the midterms......
INFO:surv_ai:......Retrieving Washington Post article with title Opinion | The GOP midterm wave is set — and Democrats can’t do anything about it......
INFO:surv_ai:......Retrieving Washington Post article with title Voting is surging in Georgia despite controversial new election law......
INFO:surv_ai:......Retrieving www.nytimes.com article with title Election night forecast: Mostly early calls, with the possibility of delays in tight races.......
INFO:surv_ai:......Retrieving Washington Post article with title The fight for

CancelledError: 

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime
from matplotlib import pyplot

COLOR_MODEL = "#69b3a2"
COLOR_538 = "#3399e6"
  
variables = model.get_plot_variables(results)


# plotting the points 
pyplot.plot([datetime(int(r[0:4]), int(r[5:7]), int(r[8:10])) for r in variables[0]], variables[1], color=COLOR_MODEL, label="surv_ai Model")
pyplot.plot([datetime(int(r[0:4]), int(r[4:6]), int(r[6:8])) for r in ['20220601', '20220701', '20220801', '20220901', '20221001', '20221101']], [1 - r for r in [0.60, 0.54, 0.44, 0.32, 0.32, 0.55]], color=COLOR_538, label="FiveThirtyEight.com Model")

pyplot.ylabel("Agreement (n_agents=100)")
pyplot.xlabel("Date")

pyplot.legend(loc="upper left")
  
# giving a title to my graph
pyplot.title('"Democrats are favored to maintain control of the Senate in the 2022 November Midterm elections."')
  
# function to show the plot
pyplot.show()

In [4]:
from surv_ai import (
    GPTClient,
    Model,
    ToolBelt,
    GoogleCustomSearchTool,
    Knowledge,
    Survey,
    SurveyParameter
)
client = GPTClient(os.environ["OPEN_AI_API_KEY"])

def build_parameter(date_range: tuple[str, str]):
    tool_belt = ToolBelt(
        tools=[
            GoogleCustomSearchTool(
                google_api_key=os.environ["GOOGLE_API_KEY"],
                google_search_engine_id=os.environ["GOOGLE_SEARCH_ENGINE_ID"],
                n_pages=20,
                start_date=date_range[0],
                end_date=date_range[1]
            ),
        ],
    )
    base_knowledge = [
        Knowledge(
            text=f"It is currently {date_range[0]}. The included articles were published between {date_range[0]} and {date_range[1]}",
            source="Additional context",
        ),
    ]
    return SurveyParameter(
        independent_variable=date_range[1],
        kwargs={
            "client": client,
            "n_agents": 100,
            "max_knowledge_per_agent":20,
            "max_concurrency": 10,
            "tool_belt": tool_belt,
            "base_knowledge": base_knowledge,
        },
    )

date_ranges = [
    ('2021-09-01', '2022-01-01'),
    ('2022-01-01', '2022-03-01'),
    ('2022-03-01', '2022-06-01'),
    ('2022-06-01', '2022-09-01'),
    ('2022-09-01', '2023-01-01'),
    ('2023-01-01', '2023-03-01'),
    ('2023-03-01', '2023-06-01'),
]

model = Model(
    Survey,
    parameters=[build_parameter(date_range) for date_range in date_ranges],
)

results = await model.build(
    "The United States economy looks like it is heading for a recession.",
)

INFO:surv_ai:...Using tool: SEARCH("Is the United States economy heading for a recession?")...
INFO:surv_ai:......Retrieving www.nytimes.com article with title How the 2020s Economy Could Resemble the 1980s (Published 2021)......
INFO:surv_ai:......Retrieving The Economist article with title Is the world economy going back to the 1970s?......
INFO:surv_ai:......Retrieving The Economist article with title Latin America is in danger of going back to the old normal......
INFO:surv_ai:......Retrieving The Economist article with title Enter third-wave economics......
INFO:surv_ai:......Retrieving www.nytimes.com article with title How America’s Pandemic Economic Response Fought the Last War (Published 2021)......
INFO:surv_ai:......Retrieving Washington Post article with title U.S. default this fall would cost 6 million jobs, wipe out $15 trillion in wealth, study says......
INFO:surv_ai:......Retrieving The Economist article with title Governments are not going to stop getting bigger......

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime
from matplotlib import pyplot

# results_1 = results

COLOR_MODEL = "#69b3a2"
COLOR_CCI = "#3399e6"
  
variables = model.get_plot_variables(results_1)
fig, ax1 = plt.subplots()
fig.set_size_inches(10.5, 5)

ax2 = ax1.twinx()

# plotting the points 
ax1.plot([datetime(int(r[0:4]), int(r[5:7]), int(r[8:10])) for r in variables[0]], [1 - r for r in variables[1]], color=COLOR_MODEL, label="Complement of surv_ai Model")
pyplot.plot([*[datetime(2022,i + 1,1) for i in range(12)], *[datetime(2023,i + 1,1) for i in range(5)]], [1.23, 1.05, 0.68, 0.68, 0.82, 0.72, 0.04, -0.4, -0.28, -0.37, -0.6, -1.2, -0.97, -1.24, -0.98, -1.18, -1.21], color=COLOR_CCI, label="Yield Spread (10y - 1y)")

ax1.set_ylabel("Agreement (n_agents=100)")
pyplot.xlabel("Date")
ax2.set_ylabel("Yield Spread (10y - 1y)")

ax1.legend(loc="upper left")
ax2.legend(loc="upper right")
  
# giving a title to my graph
pyplot.title('"The United States economy looks like it is heading for a recession."')
  
# function to show the plot
pyplot.show()

In [3]:
from surv_ai import (
    GPTClient,
    Model,
    ToolBelt,
    GoogleCustomSearchTool,
    Knowledge,
    Survey,
    SurveyParameter
)

client = GPTClient(os.environ["OPEN_AI_API_KEY"])

def build_parameter(news_source: str):
    tool_belt = ToolBelt(
        tools=[
            GoogleCustomSearchTool(
                google_api_key=os.environ["GOOGLE_API_KEY"],
                google_search_engine_id=os.environ["GOOGLE_SEARCH_ENGINE_ID"],
                n_pages=20,
                start_date="2023-05-01",
                end_date="2023-06-01",
                only_include_sources=[news_source]
            ),
        ],
    )
    base_knowledge = [
        Knowledge(
            text=f"It is currently 2023-06-01. The included articles were published between 2023-05-01 and 2023-06-01",
            source="Additional context",
        ),
    ]
    return SurveyParameter(
        independent_variable=news_source,
        kwargs={
            "client": client,
            "n_agents": 100,
            "max_knowledge_per_agent":10,
            "max_concurrency": 10,
            "tool_belt": tool_belt,
            "base_knowledge": base_knowledge,
        },
    )

news_sources = [
    "nytimes.com",
    "cnn.com",
    "wsj.com",
    "foxnews.com",
]

model = Model(
    Survey,
    parameters=[build_parameter(news_source) for news_source in news_sources],
)
results = await model.build(
    "Republicans are responsible for the impending debt ceiling crisis."
)


INFO:surv_ai:Exceeded model rate limit: attempting backoff...
INFO:surv_ai:...Using tool: SEARCH("Republicans responsible for debt ceiling crisis")...
ERROR:surv_ai:Could not retrieve all articles.
Traceback (most recent call last):
  File "/Users/danielbalsam/super_intelligence/SuperIntelligence/surv_ai/lib/tools/query/google_custom_search.py", line 86, in _search
    new_records = data["items"]
                  ~~~~^^^^^^^^^
KeyError: 'items'
INFO:surv_ai:......Retrieving www.nytimes.com article with title Finger-Pointing Won’t Save Anyone if Default Leads to Economic Collapse......
INFO:surv_ai:......Retrieving www.nytimes.com article with title Biden and McCarthy Set to Resume Negotiations on Debt Limit......
INFO:surv_ai:......Retrieving www.nytimes.com article with title Opinion | The Cowardice of the Deficit Scolds......
INFO:surv_ai:......Retrieving www.nytimes.com article with title In Debt Limit Talks, Biden and Republicans Start Far Apart......
INFO:surv_ai:......Retrieving

In [ ]:
from matplotlib import pyplot

variables = model.get_plot_variables(results)
pyplot.scatter(*variables)
pyplot.ylabel("Agreement (n_agents=100)")
pyplot.title("Republicans are responsible for the impending debt ceiling crisis.")
pyplot.show()

In [ ]:
from surv_ai import (
    GPTClient,
    AnthropicClient,
    Model,
    ToolBelt,
    GoogleCustomSearchTool,
    Knowledge,
    Survey,
    SurveyParameter,
    LargeLanguageModelClientInterface
)

clients = [AnthropicClient(os.environ["ANTHROPIC_API_KEY"]), GPTClient(os.environ["OPEN_AI_API_KEY"])]

def build_parameter(client: LargeLanguageModelClientInterface):
    tool_belt = ToolBelt(
        tools=[
            GoogleCustomSearchTool(
                os.environ["GOOGLE_API_KEY"],
                os.environ["GOOGLE_SEARCH_ENGINE_ID"],
                n_pages=20,
                start_date="2023-01-01",
                end_date="2024-05-01",
                max_concurrency=3,
            )
        ],
    )
    base_knowledge = [
        Knowledge(
            text=f"It is currently 2023-06-01. The included articles were published between 2023-01-01 and 2023-06-01",
            source="Additional context",
        ),
    ]
    return SurveyParameter(
        independent_variable=client.__class__.__name__,
        kwargs={
            "client": client,
            "n_agents": 100,
            "max_knowledge_per_agent":20,
            "max_concurrency": 3,
            "tool_belt": tool_belt,
            "base_knowledge": base_knowledge,
        },
    )

model = Model(
    Survey,
    parameters=[build_parameter(client) for client in clients],
)
results = await model.build(
    "OpenAI has been irresponsible in their handling of AI technology."
)

In [ ]:
from matplotlib import pyplot

variables = model.get_plot_variables(results)
pyplot.scatter(*variables)
pyplot.ylabel("Agreement (n_agents=100)")
pyplot.title("OpenAI has been irresponsible in their handling of AI technology.")
pyplot.show()

In [3]:
from surv_ai import (
    GPTClient,
    ToolBelt,
    GoogleCustomSearchTool,
    Knowledge,
    Survey,
)
client = GPTClient(os.environ["OPEN_AI_API_KEY"])

survey = Survey(
    client=client,
    tool_belt=ToolBelt(
        client,
        tools=[
            GoogleCustomSearchTool(
                client,
                os.environ["GOOGLE_API_KEY"],
                os.environ["GOOGLE_SEARCH_ENGINE_ID"],
                start_date="2023-01-01",
                end_date="2023-05-23",
                n_pages=10,
            )
        ]
    ),
    max_knowledge_per_agent=3,
    n_agents=10,
    base_knowledge=[
        Knowledge(
            text="It is currently 2023/05/23, all the articles are from 2023.",
            source="Additional context",
        ),
    ],
)
 
await survey.conduct(
    "The price of Bitcoin will increase over the next 30 days",
)

INFO:surv_ai:...Using tool: SEARCH('Bitcoin price prediction next 30 days')...
INFO:surv_ai:......Retrieving coincodex.com article with title Bitcoin (BTC) Price Prediction | CoinCodex......
INFO:surv_ai:......Retrieving Finbold article with title Machine learning algorithm sets Bitcoin price for April 30, 2023......
INFO:surv_ai:......Retrieving changelly.com article with title Cardano (ADA) Price Prediction 2023 2024 2025 2026 - 2030......
INFO:surv_ai:......Retrieving Forbes article with title Bitcoin At $1 Million In 90 Days And Dollar Destroyed—Huge Crypto Price Prediction Bet Fueled By Bank Crisis And Hyperinflation......
INFO:surv_ai:......Retrieving changelly.com article with title Dogecoin (DOGE) Price Prediction 2023 2024 2025 2026 - 2030......
INFO:surv_ai:......Retrieving coincodex.com article with title Dogecoin (DOGE) Price Prediction | CoinCodex......
INFO:surv_ai:......Retrieving changelly.com article with title Solana (SOL) Price Prediction 2023 2024 2025 2026 - 2030..